# Disclaimer & Copyright

Copyright 2024 Forusone : shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Vector DB - BigQuery using Google Gecko Embedding model.

* This notebook explains how to use BigQuery as a Vector database using Custom Embedding model.
* Reference
  * https://cloud.google.com/bigquery/docs/vector-search-intro?hl=ko
  * https://api.python.langchain.com/en/stable/embeddings/langchain_core.embeddings.Embeddings.html#langchain_core.embeddings.Embeddings

* Data reference
  * https://www.data.go.kr/data/15069932/fileData.do



# Configuration
## Install python packages
* Vertex AI SDK for Python
  * https://cloud.google.com/python/docs/reference/aiplatform/latest

In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 15.9 MB/s eta 0:00:00


In [8]:
%pip install --upgrade --quiet langchain_google_vertexai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 15.5 MB/s eta 0:00:00


In [3]:
from IPython.display import display, Markdown

## Authentication to access to the GCP & Google drive

* Use OAuth to access the GCP environment.
 * Refer to the authentication methods in GCP : https://cloud.google.com/docs/authentication?hl=ko

In [4]:
#  For only colab to authenticate to get an access to the GCP.
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

* Mount to the google drive to access the .ipynb files in the repository.



In [5]:
# To access contents in Google drive

if "google.colab" in sys.modules:
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


# Execute the example
## Set the environment on GCP Project
* Configure project information
  * Model name : LLM model name : https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models
  * Project Id : prodect id in GCP
  * Region : region name in GCP

In [6]:
MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

### Vertex AI initialization
Configure Vertex AI and access to the foundation model.
* Vertex AI initialization : aiplatform.init(..)
  * https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization

In [9]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
import vertexai.preview.generative_models as generative_models

# Initalizate the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=REGION)

# Access to the generative model.
model = GenerativeModel(MODEL_NAME)

## Configure Gecko Embedding
* https://api.python.langchain.com/en/v0.0.339/embeddings/langchain.embeddings.vertexai.VertexAIEmbeddings.html#

In [10]:
from langchain_google_vertexai import VertexAIEmbeddings

EBEDDING_MODEL = "textembedding-gecko-multilingual@latest"

embedding = VertexAIEmbeddings(
    model_name=EBEDDING_MODEL, project=PROJECT_ID
)

## Create dataset and table in BigQuery

In [12]:
from google.cloud import bigquery

DATASET = "vector_db_gecko"
TABLE = "vector_table_gecko"

client = bigquery.Client(project=PROJECT_ID, location=REGION)
client.create_dataset(dataset=DATASET, exists_ok=True)


Dataset(DatasetReference('ai-hangsik', 'vector_db_gecko'))

## Vector Search environment

In [13]:

from langchain.vectorstores.utils import DistanceStrategy
from langchain_community.vectorstores import BigQueryVectorSearch

table = BigQueryVectorSearch(
    project_id=PROJECT_ID,
    dataset_name=DATASET,
    table_name=TABLE,
    location=REGION,
    embedding=embedding,

    #https://api.python.langchain.com/en/stable/vectorstores/langchain_community.vectorstores.utils.DistanceStrategy.html#langchain_community.vectorstores.utils.DistanceStrategy
    distance_strategy=DistanceStrategy.COSINE

)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `BigQueryVectorSearch` was deprecated in LangChain 0.0.33 and will be removed in 0.3.0. An updated version of the class exists in the langchain-google-community package and should be used instead. To use it run `pip install -U langchain-google-community` and import as `from langchain_google_community import BigQueryVectorSearch`.
  warn_deprecated(


## Read the text data that will be vectorized.

In [14]:
import pandas as pd

file_path = '/content/drive/MyDrive/projects/google_gen_ai_sample/contents/text/legal_terminology.csv'

legal_terms = pd.read_csv(file_path,sep=",", encoding='utf-8-sig')
legal_terms

,용어명,설명
0,가등기,"부동산의 권리관계는 등기를 통하여 표시하고, 이와 같이 등기된 권리관계는 외부에 표..."
1,가등기권리자,가등기권리자란 건물의 소유자에게 돈을 빌려 주거나 건물의 매매를 약속 하기 위해 임...
2,가등기권자,가등기권리자란 건물의 소유자에게 돈을 빌려 주거나 건물의 매매를 약속 하기 위해 임...
3,매도인,"매도인이란 물건을 파는 사람이란 뜻으로, 부동산 임대차상에서는 건물이나 집 등을 판..."
4,가압류,가압류란 돈을 갚아야 하는 사람이 재산을 숨기거나 팔아 버릴 경우 돈을 받지 못할 ...
...,...,...
1018,특수지역권,특수지역권이란 어느 지역의 사람들이 집합체의 관계로 각자가 어떤 토지에서 초목과 야...
1019,필요비,"필요비란 임차인이 수도 설비나 전기 시설, 보일러 설치 등과 같이 임차물의 보존을 ..."
1020,하자없는 점유,하자없는 점유란 점유자가 점유하고 있는 물건의 권리자가 있는 것을 몰랐거나 알 수 ...
1021,하자있는 점유,"하자있는 점유란 점유자가 점유하고 있는 물건의 권리자가 있는 것을 알면서도, 그 점..."


## Embedding and store the vector data into BigQuery

In [15]:
import json

all_texts = legal_terms['설명'].to_list()
metadatas = [ {'Term': row['용어명'] } for idx, row in legal_terms.iterrows()]
table.add_texts(all_texts, metadatas=metadatas)


['d9392a92ed954ab194fbb98bd4c76a6e',
 '0cc0d78427344f0a9d8ef74583b814c3',
 '8076827238694c2daa1659700e3d0c41',
 '7df5fcee5b224ea79da260e448a2e6ab',
 '5d5ce9da2bda458d80105539efec8312',
 'fd99c48bd209454fac76c4d17b706f32',
 '276607ae0e984ae2affc66b4ec6f5c82',
 'fca688f99d324061a7dfcb20a50c6ecf',
 '5bfff5fb9b734c638c60d62ddca574d6',
 '9c6a7cbbb23048b28f9dbae6c55cba58',
 'b72608831c3d41fa84225216ab0c9f3f',
 'd77d2a0bd6ad4fe99d0b7582b0752c40',
 '3039eecff8344815ab9ea73422d2c90a',
 '3ec8524c29d34f4bbe9cae104bc1ce0d',
 'b467cbde505c4ed1b36d62a948fa98a6',
 'a4c5e82d26254467a2801e26e90f6c33',
 'c63704187687440085da0df2c46ab892',
 '37cca53dfc2e4012b42e59257d853bef',
 '1b95808201a443ddb57914681a1a2fa2',
 '96be2193eaee45ed9f3369c4e4566679',
 '9475dfa04d0f48688143c9de5492100f',
 '7d62ddbdff014a6ca3607c01044efbef',
 '71a2502f04834e8eabd380f7259afeee',
 'c29089bd970642e1876b5c91582aeb1d',
 '190bf367626a45878a063131cc3bdea7',
 '380e97a86af54271ae80eb87d28820ac',
 'a757346c8ac0449396ed1445fa5bbb35',
 

## Vector Search
### Search with string

In [16]:
import time
s = time.time()
query = "가등기권리자란?"

docs = table.similarity_search(query, k=5)
for doc in docs:
  print(f" {doc.metadata['Term']} - {doc.page_content}" )

e = time.time() - s
print(e)

 가등기권리자 - 가등기권리자란 건물의 소유자에게 돈을 빌려 주거나 건물의 매매를 약속 하기 위해 임시등기인 가등기를 설정하는 사람을 의미합니다.
 가등기권자 - 가등기권리자란 건물의 소유자에게 돈을 빌려 주거나 건물의 매매를 약속 하기 위해 임시등기인 가등기를 설정하는 사람을 의미합니다.
 가등기처분 - 가등기처분이란 상대방이 가등기 신청에 협조하지 않는 경우 법원의 명령에 따라 일방적으로 행하는 것을 의미합니다.
 가등기 - 부동산의 권리관계는 등기를 통하여 표시하고, 이와 같이 등기된 권리관계는 외부에 표시되기 때문에 제3자에게도 대항할 수 있게 됩니다. 이 때, 권리관계가 확정되지 않는 등으로 등기를 할 수 없을 경우에 임시로 하는 등기를 가등기라 합니다.
 가등기담보등에관한법률 - 가등기담보등에관한법률이란 금전을 빌리면서 채무자가 빌린 금전 대신에 다른 재산을 줄 것을 약속한 경우 그 재산의 가액에서 빌린 금액을 공제한 나머지 금전을 채권자가 반드시 채무자에게 돌려주게 함으로써 채무자를 보호할 목적으로 제정된 법률입니다.
2.4139671325683594


### Search with vector
* https://api.python.langchain.com/en/stable/vectorstores/langchain_community.vectorstores.bigquery_vector_search.BigQueryVectorSearch.html#langchain_community.vectorstores.bigquery_vector_search.BigQueryVectorSearch.similarity_search

In [17]:
query_vector = embedding.embed_query(query)
docs = table.similarity_search_by_vector(query_vector, k=5)
for doc in docs:
  print(f" {doc.metadata['Term']} - {doc.page_content}" )

 가등기권리자 - 가등기권리자란 건물의 소유자에게 돈을 빌려 주거나 건물의 매매를 약속 하기 위해 임시등기인 가등기를 설정하는 사람을 의미합니다.
 가등기권자 - 가등기권리자란 건물의 소유자에게 돈을 빌려 주거나 건물의 매매를 약속 하기 위해 임시등기인 가등기를 설정하는 사람을 의미합니다.
 가등기처분 - 가등기처분이란 상대방이 가등기 신청에 협조하지 않는 경우 법원의 명령에 따라 일방적으로 행하는 것을 의미합니다.
 가등기 - 부동산의 권리관계는 등기를 통하여 표시하고, 이와 같이 등기된 권리관계는 외부에 표시되기 때문에 제3자에게도 대항할 수 있게 됩니다. 이 때, 권리관계가 확정되지 않는 등으로 등기를 할 수 없을 경우에 임시로 하는 등기를 가등기라 합니다.
 가등기담보등에관한법률 - 가등기담보등에관한법률이란 금전을 빌리면서 채무자가 빌린 금전 대신에 다른 재산을 줄 것을 약속한 경우 그 재산의 가액에서 빌린 금액을 공제한 나머지 금전을 채권자가 반드시 채무자에게 돌려주게 함으로써 채무자를 보호할 목적으로 제정된 법률입니다.


### Search with relevant score

In [18]:
tuples = table.similarity_search_with_relevance_scores(query, k=5)
context ={}

for tp in tuples:
    context[tp[1]] = tp[0].metadata["Term"] + " : " + tp[0].page_content
context

{0.879630965993185: '가등기권자 : 가등기권리자란 건물의 소유자에게 돈을 빌려 주거나 건물의 매매를 약속 하기 위해 임시등기인 가등기를 설정하는 사람을 의미합니다.',
 0.8683882740927813: '가등기처분 : 가등기처분이란 상대방이 가등기 신청에 협조하지 않는 경우 법원의 명령에 따라 일방적으로 행하는 것을 의미합니다.',
 0.8459579552373175: '가등기 : 부동산의 권리관계는 등기를 통하여 표시하고, 이와 같이 등기된 권리관계는 외부에 표시되기 때문에 제3자에게도 대항할 수 있게 됩니다. 이 때, 권리관계가 확정되지 않는 등으로 등기를 할 수 없을 경우에 임시로 하는 등기를 가등기라 합니다.',
 0.8382316908513732: '가등기담보등에관한법률 : 가등기담보등에관한법률이란 금전을 빌리면서 채무자가 빌린 금전 대신에 다른 재산을 줄 것을 약속한 경우 그 재산의 가액에서 빌린 금액을 공제한 나머지 금전을 채권자가 반드시 채무자에게 돌려주게 함으로써 채무자를 보호할 목적으로 제정된 법률입니다.'}

### Max marginal relavant search
* https://api.python.langchain.com/en/stable/vectorstores/langchain_community.vectorstores.bigquery_vector_search.BigQueryVectorSearch.html#langchain_community.vectorstores.bigquery_vector_search.BigQueryVectorSearch.max_marginal_relevance_search

In [19]:
docs = table.max_marginal_relevance_search(query= query,
                                           k=5,
                                           fetch_k = 30,
                                           lambda_mult = 0.5,
                                           brute_force = True
                                           )
for doc in docs:
  print(doc.page_content)

가등기권리자란 건물의 소유자에게 돈을 빌려 주거나 건물의 매매를 약속 하기 위해 임시등기인 가등기를 설정하는 사람을 의미합니다.
가등기처분이란 상대방이 가등기 신청에 협조하지 않는 경우 법원의 명령에 따라 일방적으로 행하는 것을 의미합니다.
등기의무자의 권리에 관한 등기필증 또는 등기완료의 통지서가 멸실된 경우에는 등기의무자 또는 법정대리인이 직접 등기소에 출석하여 등기관이 등기의무자가 본인인지를 직접 확인하고 확인조서를 작성해야 합니다.
법정이자란 법률상 인정되어 있는 이자를 말합니다. 법정이자의 이율은 민법상의 채무에 대하여는 연 5분, 상법상의 채무에 대하여는 연 6분으로 규정하고 있습니다.
본등기란 임시등기인 가등기와 다르게 권리 관계의 효력을 완전하게 발생시키는 본래 등기를 의미합니다.
